In [31]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import sys

from tqdm import tqdm

def get_inten(ctype, frac, mode, angle):
    
    I = np.zeros((512 * 16, 120))
    
    specs = glob.glob('./runs/nessy/G2V/matthias_ext/' + ctype + '/' + str(frac) + '/' + mode + '/' + str(angle) + '/*')
        
    idx = []
        
    for i in range(0, len(specs), 4): idx.append(i)
    
    for i in tqdm(range(len(idx)), desc = ctype + ' ' + mode + ' ' + str(angle)):
        
        if os.path.getsize(specs[idx[i]]) > 0: I[i, :] = np.loadtxt(specs[idx[i]])
            
    return I

def read_nessy_spec(angles):
    
    I0lf = np.load('specs_3D.npz')['I0l'].reshape(512 * 512, 120)
    ISlf = np.load('specs_3D.npz')['ISl'].reshape(512 * 512, 120)
    I3lf = np.load('specs_3D.npz')['I3l'].reshape(512 * 512, 120)
    I0nf = np.load('specs_3D.npz')['I0n'].reshape(512 * 512, 120)
    ISnf = np.load('specs_3D.npz')['ISn'].reshape(512 * 512, 120)
    I3nf = np.load('specs_3D.npz')['I3n'].reshape(512 * 512, 120)

    I0l = np.zeros((512 * 16, 5, 120))
    ISl = np.zeros((512 * 16, 5, 120))
    I3l = np.zeros((512 * 16, 5, 120))
    I0n = np.zeros((512 * 16, 5, 120))
    ISn = np.zeros((512 * 16, 5, 120))
    I3n = np.zeros((512 * 16, 5, 120))

    I0l[:, 4, :] = I0lf[0::32, :]
    ISl[:, 4, :] = ISlf[0::32, :]
    I3l[:, 4, :] = I3lf[0::32, :]
    I0n[:, 4, :] = I0nf[0::32, :]
    ISn[:, 4, :] = ISnf[0::32, :]
    I3n[:, 4, :] = I3nf[0::32, :]
    
    for j in range(len(angles)):
        
        I0l[:, j, :] = get_inten('hydro', 8, 'lte',  angles[j])
        ISl[:, j, :] = get_inten('ssd',   8, 'lte',  angles[j])
        I3l[:, j, :] = get_inten('300G',  8, 'lte',  angles[j])
        
        I0n[:, j, :] = get_inten('hydro', 8, 'nlte', angles[j])
        ISn[:, j, :] = get_inten('ssd',   8, 'nlte', angles[j])
        I3n[:, j, :] = get_inten('300G',  8, 'nlte', angles[j])
                
    return I0l, ISl, I3l, I0n, ISn, I3n

#I0l, ISl, I3l, I0n, ISn, I3n = read_nessy_spec([2, 4, 6, 8])

#np.savez('spec_nessy_G2V_matthias_ext_32', I0l = I0l, ISl = ISl, I3l = I3l, I0n = I0n, ISn = ISn, I3n = I3n)

I0l = np.load('spec_nessy_G2V_matthias_ext_32.npz')['I0l']
ISl = np.load('spec_nessy_G2V_matthias_ext_32.npz')['ISl']
I3l = np.load('spec_nessy_G2V_matthias_ext_32.npz')['I3l']
I0n = np.load('spec_nessy_G2V_matthias_ext_32.npz')['I0n']
ISn = np.load('spec_nessy_G2V_matthias_ext_32.npz')['ISn']
I3n = np.load('spec_nessy_G2V_matthias_ext_32.npz')['I3n']

I0l_m = np.zeros((5, 120))
ISl_m = np.zeros((5, 120))
I3l_m = np.zeros((5, 120))
I0n_m = np.zeros((5, 120))
ISn_m = np.zeros((5, 120))
I3n_m = np.zeros((5, 120))

plt_mu = [4, 3, 2, 1, 0]

for i in plt_mu:

    for j in range(120):
        
        nz0l = np.nonzero(I0l[:, i, j])
        nzSl = np.nonzero(ISl[:, i, j])
        nz3l = np.nonzero(I3l[:, i, j])
        nz0n = np.nonzero(I0n[:, i, j])
        nzSn = np.nonzero(ISn[:, i, j])
        nz3n = np.nonzero(I3n[:, i, j])
    
        I0l_m[i, j] = np.mean(I0l[nz0l, i, j])
        ISl_m[i, j] = np.mean(ISl[nzSl, i, j])
        I3l_m[i, j] = np.mean(I3l[nz3l, i, j])
        I0n_m[i, j] = np.mean(I0n[nz0n, i, j])
        ISn_m[i, j] = np.mean(ISn[nzSn, i, j])
        I3n_m[i, j] = np.mean(I3n[nz3n, i, j])

w = np.arange(1805, 3005, 10) / 10

np.savez('spec_nessy_G2V_matthias_ext_32_mean', I0l = I0l_m, ISl = ISl_m, I3l = I3l_m, I0n = I0n_m, ISn = ISn_m, I3n = I3n_m)

In [21]:
import itertools

mu = [0.2, 0.4, 0.6, 0.8, 1.0]

pdf_names = ''

for i in tqdm(plt_mu):
    
    plt.close('all')
    
    fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (15, 10))
    
#   plt.rcParams['font.size'] = 15
    
    fig.tight_layout()
    
    fig.suptitle(r'$\mu$ = ' + str(mu[i]), y = 1.01)
    
    nz0l = np.nonzero(I0l[:, i, 119])
    nzSl = np.nonzero(ISl[:, i, 119])
    nz3l = np.nonzero(I3l[:, i, 119])
    nz0n = np.nonzero(I0n[:, i, 119])
    nzSn = np.nonzero(ISn[:, i, 119])
    nz3n = np.nonzero(I3n[:, i, 119])

    for k in range(0, len(nz0l[0]), 200): ax[0, 0].plot(w, I0l[k, i, :], color = 'gray')
    for k in range(0, len(nz0n[0]), 200): ax[0, 1].plot(w, I0n[k, i, :], color = 'gray')
    for k in range(0, len(nzSl[0]), 200): ax[1, 0].plot(w, ISl[k, i, :], color = 'gray')
    for k in range(0, len(nzSn[0]), 200): ax[1, 1].plot(w, ISn[k, i, :], color = 'gray')
    for k in range(0, len(nz3l[0]), 200): ax[2, 0].plot(w, I3l[k, i, :], color = 'gray')
    for k in range(0, len(nz3n[0]), 200): ax[2, 1].plot(w, I3n[k, i, :], color = 'gray')

    ax[0, 0].plot(w, I0l_m[i, :], linewidth = 2, color = 'yellow')
    ax[0, 1].plot(w, I0n_m[i, :], linewidth = 2, color = 'yellow')
    ax[1, 0].plot(w, ISl_m[i, :], linewidth = 2, color = 'yellow')
    ax[1, 1].plot(w, ISn_m[i, :], linewidth = 2, color = 'yellow')
    ax[2, 0].plot(w, I3l_m[i, :], linewidth = 2, color = 'yellow')
    ax[2, 1].plot(w, I3n_m[i, :], linewidth = 2, color = 'yellow')
    
    ax[0, 0].set_title('LTE')
    ax[0, 1].set_title('NLTE')

    ax[0, 0].text(270, 5e-12, 'hydro')
    ax[0, 1].text(270, 5e-12, 'hydro')
    ax[1, 0].text(270, 5e-12, 'ssd')
    ax[1, 1].text(270, 5e-12, 'ssd')
    ax[2, 0].text(270, 5e-12, '300G')
    ax[2, 1].text(270, 5e-12, '300G')

    ax[2, 0].set_xlabel('Wavelength, nm')
    ax[2, 1].set_xlabel('Wavelength, nm')
    ax[0, 0].set_ylabel('Intensity, cgs')
    ax[1, 0].set_ylabel('Intensity, cgs')
    ax[2, 0].set_ylabel('Intensity, cgs')

    for p, q in itertools.product(range(3), range(2)):

        ax[p, q].set_yscale('log')
        ax[p, q].set_xlim(180, 300)
        ax[p, q].set_ylim(1e-12, 1e-4)
        
    pdf_name = './angle_res/allrays_' + str(i) + '.pdf'
        
    plt.savefig(pdf_name, bbox_inches = 'tight')
    
    pdf_names += pdf_name + ' '
    
os.system('pdftk ' + pdf_names + ' output ./angle_res/allrays.pdf')

os.system('rm ./angle_res/allrays_*.pdf')
    
#    plt.show()

100%|██████████| 5/5 [00:23<00:00,  4.67s/it]


0

In [22]:
plt.close('all')

fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (15, 10))

for i in plt_mu:
    
#   plt.rcParams['font.size'] = 15
    
    fig.tight_layout()
    
#    fig.suptitle(r'$\mu$ = ' + str(mu[i]), y = 1.01)

    ax[0, 0].plot(w, I0l_m[i, :], linewidth = 2, label = r'$\mu$ = ' + str(mu[i]))
    ax[0, 1].plot(w, I0n_m[i, :], linewidth = 2, label = r'$\mu$ = ' + str(mu[i]))
    ax[1, 0].plot(w, ISl_m[i, :], linewidth = 2, label = r'$\mu$ = ' + str(mu[i]))
    ax[1, 1].plot(w, ISn_m[i, :], linewidth = 2, label = r'$\mu$ = ' + str(mu[i]))
    ax[2, 0].plot(w, I3l_m[i, :], linewidth = 2, label = r'$\mu$ = ' + str(mu[i]))
    ax[2, 1].plot(w, I3n_m[i, :], linewidth = 2, label = r'$\mu$ = ' + str(mu[i]))
    
    ax[0, 0].set_title('LTE')
    ax[0, 1].set_title('NLTE')

    ax[0, 0].text(270, 5e-9, 'hydro')
    ax[0, 1].text(270, 5e-9, 'hydro')
    ax[1, 0].text(270, 5e-9, 'ssd')
    ax[1, 1].text(270, 5e-9, 'ssd')
    ax[2, 0].text(270, 5e-9, '300G')
    ax[2, 1].text(270, 5e-9, '300G')

    ax[2, 0].set_xlabel('Wavelength, nm')
    ax[2, 1].set_xlabel('Wavelength, nm')
    ax[0, 0].set_ylabel('Intensity, cgs')
    ax[1, 0].set_ylabel('Intensity, cgs')
    ax[2, 0].set_ylabel('Intensity, cgs')

    for p, q in itertools.product(range(3), range(2)):

        ax[p, q].set_yscale('log')
        ax[p, q].set_xlim(180, 300)
        ax[p, q].set_ylim(1e-9, 1e-5)
        
leg = ax[1, 0].legend(framealpha = 1, loc = 2, handletextpad = 1, prop = {'size': 12.5})
        
plt.savefig('./angle_res/allmeans.pdf', bbox_inches = 'tight')
    
#plt.show()

In [23]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (15, 2.5))

for i in plt_mu:
    
#    fig.suptitle(r'$\mu$ = ' + str(mu[i]), y = 1.01)

    ax[0].plot(w, I0l_m[i, :] / ISl_m[i, :], linewidth = 2, label = r'$\mu$ = ' + str(mu[i]))
    ax[1].plot(w, I0n_m[i, :] / ISn_m[i, :], linewidth = 2, label = r'$\mu$ = ' + str(mu[i]))

    ax[0].set_title('LTE')
    ax[1].set_title('NLTE')

    ax[0].set_xlabel('Wavelength, nm')
    ax[1].set_xlabel('Wavelength, nm')
    ax[0].set_ylabel('hydro / ssd')
    
    ax[0].set_xlim(180, 300)
    ax[1].set_xlim(180, 300)
    
#    ax[0].set_ylim(0.85, 1.00)
#    ax[1].set_ylim(0.85, 1.00)

    ax[0].set_ylim(0.65, 1.25)
    ax[1].set_ylim(0.65, 1.25)
    
leg = ax[1].legend(framealpha = 1, loc = 4, handletextpad = 1, prop = {'size': 12.5})
    
plt.savefig('./angle_res/hydro_to_ssd.pdf', bbox_inches = 'tight')

#plt.show()

In [34]:
Il_fal_q = np.zeros((9, 120))
Il_fal_f = np.zeros((9, 120))

In_fal_q = np.zeros((9, 120))
In_fal_f = np.zeros((9, 120))

Il_fal_q_tmin = np.zeros((9, 120))
Il_fal_f_tmin = np.zeros((9, 120))

In_fal_q_tmin = np.zeros((9, 120))
In_fal_f_tmin = np.zeros((9, 120))

Il_kur_q = np.zeros((9, 120))
Il_kur_f = np.zeros((9, 120))

In_kur_q = np.zeros((9, 120))
In_kur_f = np.zeros((9, 120))

idx_m = plt_mu
idx_n = [1, 3, 5, 7, 9]

for i in range(len(idx_m)):

    Il_fal_q[idx_m[i], :] = np.loadtxt('clv_lte_fal_q',  usecols = [idx_n[i]])
    Il_fal_f[idx_m[i], :] = np.loadtxt('clv_lte_fal_f',  usecols = [idx_n[i]])

    In_fal_q[idx_m[i], :] = np.loadtxt('clv_nlte_fal_q', usecols = [idx_n[i]])
    In_fal_f[idx_m[i], :] = np.loadtxt('clv_nlte_fal_f', usecols = [idx_n[i]])
    
    Il_fal_q_tmin[idx_m[i], :] = np.loadtxt('clv_lte_fal_q_tmin',  usecols = [idx_n[i]])
    Il_fal_f_tmin[idx_m[i], :] = np.loadtxt('clv_lte_fal_f_tmin',  usecols = [idx_n[i]])

    In_fal_q_tmin[idx_m[i], :] = np.loadtxt('clv_nlte_fal_q_tmin', usecols = [idx_n[i]])
    In_fal_f_tmin[idx_m[i], :] = np.loadtxt('clv_nlte_fal_f_tmin', usecols = [idx_n[i]])

    Il_kur_q[idx_m[i], :] = np.loadtxt('clv_lte_kur_q',  usecols = [idx_n[i]])
    Il_kur_f[idx_m[i], :] = np.loadtxt('clv_lte_kur_f',  usecols = [idx_n[i]])

    In_kur_q[idx_m[i], :] = np.loadtxt('clv_nlte_kur_q', usecols = [idx_n[i]])
    In_kur_f[idx_m[i], :] = np.loadtxt('clv_nlte_kur_f', usecols = [idx_n[i]])

In [36]:
pdf_names = ''

for i in tqdm(plt_mu):
    
    plt.close('all')

    fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (15, 10))
    
    fig.suptitle(r'$\mu$ = ' + str(mu[i]), y = 1.01)

    fig.tight_layout()

    ax[0, 0].plot(w, I0l_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[0, 0].plot(w, Il_fal_q_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[0, 0].plot(w, Il_fal_q[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[0, 0].plot(w, Il_kur_q[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    #---------------------------------------------------------------------------------------------------

    ax[0, 1].plot(w, I0n_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[0, 1].plot(w, In_fal_q_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[0, 1].plot(w, In_fal_q[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[0, 1].plot(w, In_kur_q[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    #---------------------------------------------------------------------------------------------------

    ax[1, 0].plot(w, ISl_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[1, 0].plot(w, Il_fal_q_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[1, 0].plot(w, Il_fal_q[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[1, 0].plot(w, Il_kur_q[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    #---------------------------------------------------------------------------------------------------

    ax[1, 1].plot(w, ISn_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[1, 1].plot(w, In_fal_q_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[1, 1].plot(w, In_fal_q[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[1, 1].plot(w, In_kur_q[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    #---------------------------------------------------------------------------------------------------

    ax[2, 0].plot(w, I3l_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[2, 0].plot(w, Il_fal_f_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[2, 0].plot(w, Il_fal_f[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[2, 0].plot(w, Il_kur_f[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    #---------------------------------------------------------------------------------------------------

    ax[2, 1].plot(w, I3n_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[2, 1].plot(w, In_fal_f_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[2, 1].plot(w, In_fal_f[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[2, 1].plot(w, In_kur_f[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')

    ax[0, 0].set_title('LTE')
    ax[0, 1].set_title('NLTE')

    ax[0, 0].text(270, 5e-8, 'hydro')
    ax[0, 1].text(270, 5e-8, 'hydro')
    ax[1, 0].text(270, 5e-8, 'ssd')
    ax[1, 1].text(270, 5e-8, 'ssd')
    ax[2, 0].text(270, 5e-8, '300G')
    ax[2, 1].text(270, 5e-8, '300G')

    ax[2, 0].set_xlabel('Wavelength, nm')
    ax[2, 1].set_xlabel('Wavelength, nm')
    ax[0, 0].set_ylabel('Intensity, cgs')
    ax[1, 0].set_ylabel('Intensity, cgs')
    ax[2, 0].set_ylabel('Intensity, cgs')

    for p, q in itertools.product(range(3), range(2)): 
    
        ax[p, q].set_yscale('log')
        
        ax[p, q].set_ylim(1e-9, 1e-5)
        ax[p, q].set_xlim(180, 300)

    leg = ax[0, 0].legend(framealpha = 1, loc = 2, handletextpad = 1, prop = {'size': 12.5})
    
    pdf_name = './angle_res/models_comp_' + str(i) + '.pdf'
        
    plt.savefig(pdf_name, bbox_inches = 'tight')
    
    pdf_names += pdf_name + ' '
    
os.system('pdftk ' + pdf_names + ' output ./angle_res/models_comp.pdf')

os.system('rm ./angle_res/models_comp_*.pdf')

#    plt.show()

100%|██████████| 5/5 [00:14<00:00,  3.00s/it]


0

In [26]:
pdf_names = ''

for i in tqdm(plt_mu):
    
    plt.close('all')

    fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (15, 10))
    
    fig.suptitle(r'$\mu$ = ' + str(mu[i]), y = 1.01)

    fig.tight_layout()

    ax[0, 0].plot(w, Il_fal_q[i, :] / I0l_m[i, :], linewidth = 2, color = 'magenta', label = 'fal')
    ax[0, 0].plot(w, Il_kur_q[i, :] / I0l_m[i, :], linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    ax[0, 0].axhline(y = 1.0, linestyle = '--', color = 'black')

    ax[0, 1].plot(w, In_fal_q[i, :] / I0n_m[i, :], linewidth = 2, color = 'magenta', label = 'fal')
    ax[0, 1].plot(w, In_kur_q[i, :] / I0n_m[i, :], linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    ax[0, 1].axhline(y = 1.0, linestyle = '--', color = 'black')

    ax[1, 0].plot(w, Il_fal_q[i, :] / ISl_m[i, :], linewidth = 2, color = 'magenta', label = 'fal')
    ax[1, 0].plot(w, Il_kur_q[i, :] / ISl_m[i, :], linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    ax[1, 0].axhline(y = 1.0, linestyle = '--', color = 'black')

    ax[1, 1].plot(w, In_fal_q[i, :] / ISn_m[i, :], linewidth = 2, color = 'magenta', label = 'fal')
    ax[1, 1].plot(w, In_kur_q[i, :] / ISn_m[i, :], linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    ax[1, 1].axhline(y = 1.0, linestyle = '--', color = 'black')

    ax[2, 0].plot(w, Il_fal_f[i, :] / I3l_m[i, :], linewidth = 2, color = 'magenta', label = 'fal')
    ax[2, 0].plot(w, Il_kur_f[i, :] / I3l_m[i, :], linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    ax[2, 0].axhline(y = 1.0, linestyle = '--', color = 'black')

    ax[2, 1].plot(w, In_fal_f[i, :] / I3n_m[i, :], linewidth = 2, color = 'magenta', label = 'fal')
    ax[2, 1].plot(w, In_kur_f[i, :] / I3n_m[i, :], linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    ax[2, 1].axhline(y = 1.0, linestyle = '--', color = 'black')

    ax[0, 0].set_title('LTE')
    ax[0, 1].set_title('NLTE')

#    ax[0, 0].text(270, 5e-8, 'hydro')
#    ax[0, 1].text(270, 5e-8, 'hydro')
#    ax[1, 0].text(270, 5e-8, 'ssd')
#    ax[1, 1].text(270, 5e-8, 'ssd')
#    ax[2, 0].text(270, 5e-8, '300G')
#    ax[2, 1].text(270, 5e-8, '300G')

    ax[2, 0].set_xlabel('Wavelength, nm')
    ax[2, 1].set_xlabel('Wavelength, nm')
#    ax[0, 0].set_ylabel('Intensity, cgs')
#    ax[1, 0].set_ylabel('Intensity, cgs')
#    ax[2, 0].set_ylabel('Intensity, cgs')
    ax[0, 0].set_ylabel('Ratio: hydro')
    ax[1, 0].set_ylabel('Ratio: ssd')
    ax[2, 0].set_ylabel('Ratio: 300G')

    for p, q in itertools.product(range(3), range(2)): 
    
#        ax[p, q].set_yscale('log')
        
#        ax[p, q].set_ylim(1e-9, 1e-5)
        ax[p, q].set_ylim(0.3, 6.3)
        ax[p, q].set_xlim(180, 300)

    leg = ax[0, 0].legend(framealpha = 1, loc = 2, handletextpad = 1, prop = {'size': 12.5})
    
    pdf_name = './angle_res/models_rat_' + str(i) + '.pdf'
        
    plt.savefig(pdf_name, bbox_inches = 'tight')
    
    pdf_names += pdf_name + ' '
    
os.system('pdftk ' + pdf_names + ' output ./angle_res/models_rat.pdf')

os.system('rm ./angle_res/models_rat_*.pdf')

#    plt.show()

100%|██████████| 5/5 [00:07<00:00,  1.53s/it]


0

In [27]:
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MultipleLocator

pdf_names = ''

for i in tqdm(plt_mu):
    
    plt.close('all')

    fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (15, 10))
    
    fig.suptitle(r'$\mu$ = ' + str(mu[i]), y = 1.01)

    fig.tight_layout()

    plt.subplots_adjust(hspace = 0.5)

    ax[0, 0].plot(w, I0l_m[i, :] * 1e+6, color = 'k', label = 'LTE')
    ax[0, 0].plot(w, I0n_m[i, :] * 1e+6, color = 'r', label = 'NLTE')
    ax[0, 1].plot(w, ISl_m[i, :] * 1e+6, color = 'k', label = 'LTE')
    ax[0, 1].plot(w, ISn_m[i, :] * 1e+6, color = 'r', label = 'NLTE')
    ax[0, 2].plot(w, I3l_m[i, :] * 1e+6, color = 'k', label = 'LTE')
    ax[0, 2].plot(w, I3n_m[i, :] * 1e+6, color = 'r', label = 'NLTE')

    ax[1, 0].plot(w, Il_fal_q[i, :] * 1e+6, color = 'k', label = 'LTE')
    ax[1, 0].plot(w, In_fal_q[i, :] * 1e+6, color = 'r', label = 'NLTE')
#    ax[1, 1].plot(w, Il_fal_q[i, :] * 1e+6, color = 'k', label = 'LTE')
#    ax[1, 1].plot(w, In_fal_q[i, :] * 1e+6, color = 'r', label = 'NLTE')
    ax[1, 2].plot(w, Il_fal_f[i, :] * 1e+6, color = 'k', label = 'LTE')
    ax[1, 2].plot(w, In_fal_f[i, :] * 1e+6, color = 'r', label = 'NLTE')

    ax[2, 0].plot(w, Il_kur_q[i, :] * 1e+6, color = 'k', label = 'LTE')
    ax[2, 0].plot(w, In_kur_q[i, :] * 1e+6, color = 'r', label = 'NLTE')
#    ax[2, 1].plot(w, Il_kur_q[i, :] * 1e+6, color = 'k', label = 'LTE')
#    ax[2, 1].plot(w, In_kur_q[i, :] * 1e+6, color = 'r', label = 'NLTE') 
    ax[2, 2].plot(w, Il_kur_f[i, :] * 1e+6, color = 'k', label = 'LTE')
    ax[2, 2].plot(w, In_kur_f[i, :] * 1e+6, color = 'r', label = 'NLTE')

    ax[0, 0].set_title('hydro')
    ax[0, 1].set_title('ssd')
    ax[0, 2].set_title('300G')

    ax[2, 0].set_xlabel('Wavelength, nm')
    ax[2, 2].set_xlabel('Wavelength, nm')
    ax[0, 0].set_ylabel('Intensity, cgs * 1e+6')
    ax[1, 0].set_ylabel('Intensity, cgs * 1e+6')
    ax[2, 0].set_ylabel('Intensity, cgs * 1e+6')

    ax[0, 0].text(270, 5e-3, 'muram')
    ax[0, 2].text(270, 3e-2, 'muram')
    ax[1, 0].text(270, 5e-3, 'fal')
    ax[1, 2].text(270, 3e-2, 'fal')
    ax[2, 0].text(270, 5e-3, 'kurucz')
    ax[2, 2].text(270, 3e-2, 'kurucz')

    for p, q in itertools.product(range(3), range(3)): 
    
        ax[p, q].set_yscale('log')
        
        ax[p, q].set_xlim(180, 300)
        ax[p, q].set_ylim(1e-3, 1e+1)


    leg = ax[0, 2].legend(framealpha = 1, loc = 2, handletextpad = 1, prop = {'size': 12.5})
    
    pdf_name = './angle_res/nlte_lte_comp_' + str(i) + '.pdf'
        
    plt.savefig(pdf_name, bbox_inches = 'tight')
    
    pdf_names += pdf_name + ' '
    
os.system('pdftk ' + pdf_names + ' output ./angle_res/nlte_lte_comp.pdf')

os.system('rm ./angle_res/nlte_lte_comp_*.pdf')

#    plt.show()

100%|██████████| 5/5 [00:19<00:00,  3.85s/it]


0

In [28]:
pdf_names = ''

for i in tqdm(plt_mu):
    
    plt.close('all')

    fig, ax = plt.subplots(nrows = 2, ncols = 1, figsize = (10, 5))
    
    #fig.suptitle(r'$\mu$ = ' + str(mu[i]), y = 1.01)
    ax[0].set_title(r'$\mu$ = ' + str(mu[i]), y = 1.01)

    ax[0].plot(w, I0n_m[i, :] / I0l_m[i, :],       color = 'b', label = 'muram hydro')
    ax[0].plot(w, ISn_m[i, :] / ISl_m[i, :],       color = 'r', label = 'muram ssd')
    ax[0].plot(w, In_fal_q[i, :] / Il_fal_q[i, :], color = 'm', label = 'fal')
    ax[0].plot(w, In_kur_q[i, :] / Il_kur_q[i, :], color = 'c', label = 'kurucz')

    ax[1].plot(w, I3n_m[i, :] / I3l_m[i, :],       color = 'b', label = 'muram 300G')
    ax[1].plot(w, In_fal_f[i, :] / Il_fal_f[i, :], color = 'm', label = 'fal')
    ax[1].plot(w, In_kur_f[i, :] / Il_kur_f[i, :], color = 'c', label = 'kurucz')

    ax[0].xaxis.set_major_locator(MultipleLocator(10))
    ax[0].yaxis.set_minor_locator(MultipleLocator(1))
    ax[1].xaxis.set_major_locator(MultipleLocator(10))
    ax[1].yaxis.set_minor_locator(MultipleLocator(1))

    ax[0].set_ylim(0.5, 11)
    ax[1].set_ylim(0.5, 11)
    
    ax[0].set_xlim(180, 300)
    ax[1].set_xlim(180, 300)

    ax[1].set_xlabel('Wavelength, nm')
    ax[0].set_ylabel('NLTE / LTE')
    ax[1].set_ylabel('NLTE / LTE')

    ax[0].text(280, 1.5, 'quiet')
    ax[1].text(280, 1.5, 'facula')

    leg = ax[0].legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 10.5})
    leg = ax[1].legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 10.5})
    
    pdf_name = './angle_res/nlte_lte_rat_' + str(i) + '.pdf'
        
    plt.savefig(pdf_name, bbox_inches = 'tight')
    
    pdf_names += pdf_name + ' '
    
os.system('pdftk ' + pdf_names + ' output ./angle_res/nlte_lte_rat.pdf')

os.system('rm ./angle_res/nlte_lte_rat_*.pdf')

#    plt.show()

100%|██████████| 5/5 [00:03<00:00,  1.35it/s]


0

In [29]:
pdf_names = ''

for i in tqdm(plt_mu):

    rcontr_n = I3n_m[i, :] / I0n_m[i, :]
    rcontr_l = I3l_m[i, :] / I0l_m[i, :]

    rcontr_n_ssd = I3n_m[i, :] / ISn_m[i, :]
    rcontr_l_ssd = I3l_m[i, :] / ISl_m[i, :]

    rcontr_n_fal = In_fal_f[i, :] / In_fal_q[i, :]
    rcontr_l_fal = Il_fal_f[i, :] / Il_fal_q[i, :]

    rcontr_n_kur = In_kur_f[i, :] / In_kur_q[i, :]
    rcontr_l_kur = Il_kur_f[i, :] / Il_kur_q[i, :]
    
    plt.close('all')

    fig, ax = plt.subplots(nrows = 3, ncols = 1, figsize = (7.5, 7.5))
    
#    fig.suptitle(r'$\mu$ = ' + str(mu[i]), y = 0.999999)
    ax[0].set_title(r'$\mu$ = ' + str(mu[i]), y = 0.999999)

    fig.tight_layout()

    plt.subplots_adjust(hspace = 0.25)

#    ax[0].plot(w, rcontr_l, color ='k', label = 'LTE (hydro)', linewidth = 3)
#    ax[0].plot(w, rcontr_n, color ='r', label = 'NLTE (hydro)', linewidth = 3)
    ax[0].plot(w, rcontr_l, color ='k', label = 'LTE', linewidth = 2)
    ax[0].plot(w, rcontr_n, color ='r', label = 'NLTE', linewidth = 2)

#    ax[0].plot(w, rcontr_l_ssd, color ='cyan',   label = 'LTE (ssd)')
#    ax[0].plot(w, rcontr_n_ssd, color ='green', label = 'NLTE (ssd)')

    ax[1].plot(w, rcontr_l_fal, color ='k', label = 'LTE')
    ax[1].plot(w, rcontr_n_fal, color ='r', label = 'NLTE')

    ax[2].plot(w, rcontr_l_kur, color ='k', label = 'LTE')
    ax[2].plot(w, rcontr_n_kur, color ='r', label = 'NLTE')

    ax[0].axhline(y = 1.0, color = 'k', linestyle = '--')
    ax[1].axhline(y = 1.0, color = 'k', linestyle = '--')
    ax[2].axhline(y = 1.0, color = 'k', linestyle = '--')

    ax[2].set_xlabel('Wavelength, nm')
    ax[0].set_ylabel('300G / {hydro,ssd}')
    ax[1].set_ylabel('facula / quiet')
    ax[2].set_ylabel('facula / quiet')

    ax[0].set_ylim(0.5, 4.0)
    ax[1].set_ylim(0.5, 4.5)
    ax[2].set_ylim(0.5, 7.0)

    ax[0].set_xlim(180, 300)
    ax[1].set_xlim(180, 300)
    ax[2].set_xlim(180, 300)

    ax[0].xaxis.set_major_locator(MultipleLocator(10))
    ax[1].xaxis.set_major_locator(MultipleLocator(10))
    ax[2].xaxis.set_major_locator(MultipleLocator(10))

    ax[0].text(250, 3.0, 'muram')
    ax[1].text(250, 3.0, 'fal')
    ax[2].text(250, 5.0, 'kurucz')

    leg = ax[0].legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 10.5})
    
    pdf_name = './angle_res/contrast_' + str(i) + '.pdf'
        
    plt.savefig(pdf_name, bbox_inches = 'tight')
    
    pdf_names += pdf_name + ' '
    
os.system('pdftk ' + pdf_names + ' output ./angle_res/contrast.pdf')

os.system('rm ./angle_res/contrast_*.pdf')

#    plt.show()

100%|██████████| 5/5 [00:03<00:00,  1.27it/s]


0